In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb

## NOTE: this model is gonna be bad... not symmetrical, inputs arbitrarily chosen, not enough training data

# reading dat
dat = pd.read_csv("MLData/combined.csv")

# modifying dat to remove character values
dat["WinAsInt"] = dat["B_Win"].transform(lambda x: int(x == "WIN"))
dat = dat.drop(axis = 1, labels = "B_Win")

In [ ]:
correlations = dat.corr()["WinAsInt"].sort_values(ascending = False)

print(correlations)

In [ ]:
# thinning out dataset
print(correlations[correlations < -0.13])
print()
print(correlations[correlations > 0.13])

In [ ]:
input_vars = correlations[correlations < -0.13].index.to_list() + correlations[correlations > 0.13].index.to_list()
training_dat = dat[input_vars]
# thinned out dataset actually substantially worse at predicting than the full one is
# training_dat = dat
print(training_dat)

# split test and training data
test_dat = training_dat.sample(frac = 0)
training_dat = training_dat.drop(test_dat.index)

# setting up for xgboost
training_label = training_dat["WinAsInt"]
training_dat = training_dat.drop(axis = 1, labels = "WinAsInt")
test_label = test_dat["WinAsInt"]
test_dat = test_dat.drop(axis = 1, labels = "WinAsInt")

dtrain = xgb.DMatrix(data = training_dat, label = training_label)
dtest = xgb.DMatrix(data = test_dat, label = test_label)

print(training_dat.shape)
print(test_dat.shape)


In [ ]:
param = {'max_depth': 2, 'eta': 0.1, 'objective': 'binary:logistic'}
param['eval_metric'] = 'auc'

num_round = 10
evallist = [(dtrain, "train"), (dtest, "test")]
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
#xgb.plot_importance(bst)

In [ ]:
prediction_data = pd.read_csv("MLData/LEC_2023_Spring_Test.csv")
teams = prediction_data[["B_Team", "R_Team"]]
prediction_data = prediction_data.drop(axis = 1, labels = ["B_Team", "R_Team"])

input_vars.remove("WinAsInt")

prediction_data = prediction_data[input_vars]

dpred = xgb.DMatrix(prediction_data)

scores = bst.predict(dpred)
teams["Small Model Scores (0.13)"] = scores

print(teams)

In [ ]:
input_vars = correlations[correlations < -0.10].index.to_list() + correlations[correlations > 0.10].index.to_list()
training_dat = dat[input_vars]
# thinned out dataset actually substantially worse at predicting than the full one is
# training_dat = dat
print(training_dat)

# split test and training data
test_dat = training_dat.sample(frac = 0)
training_dat = training_dat.drop(test_dat.index)

# setting up for xgboost
training_label = training_dat["WinAsInt"]
training_dat = training_dat.drop(axis = 1, labels = "WinAsInt")
test_label = test_dat["WinAsInt"]
test_dat = test_dat.drop(axis = 1, labels = "WinAsInt")

dtrain = xgb.DMatrix(data = training_dat, label = training_label)
dtest = xgb.DMatrix(data = test_dat, label = test_label)

print(training_dat.shape)
print(test_dat.shape)

In [ ]:
param = {'max_depth': 2, 'eta': 0.1, 'objective': 'binary:logistic'}
param['eval_metric'] = 'auc'

num_round = 30
evallist = [(dtrain, "train"), (dtest, "test")]
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
prediction_data = pd.read_csv("MLData/LEC_2023_Spring_Test.csv")
# teams = prediction_data[["B_Team", "R_Team"]]
prediction_data = prediction_data.drop(axis = 1, labels = ["B_Team", "R_Team"])

input_vars.remove("WinAsInt")

prediction_data = prediction_data[input_vars]

dpred = xgb.DMatrix(prediction_data)

scores = bst.predict(dpred)
teams["Med Model Scores (0.1)"] = scores

print(teams)

In [ ]:
input_vars = dat.columns.to_list()
training_dat = dat[input_vars]
# thinned out dataset actually substantially worse at predicting than the full one is
# training_dat = dat
print(training_dat)

# split test and training data
test_dat = training_dat.sample(frac = 0)
training_dat = training_dat.drop(test_dat.index)

# setting up for xgboost
training_label = training_dat["WinAsInt"]
training_dat = training_dat.drop(axis = 1, labels = "WinAsInt")
test_label = test_dat["WinAsInt"]
test_dat = test_dat.drop(axis = 1, labels = "WinAsInt")

dtrain = xgb.DMatrix(data = training_dat, label = training_label)
dtest = xgb.DMatrix(data = test_dat, label = test_label)

print(training_dat.shape)
print(test_dat.shape)

In [ ]:
param = {'max_depth': 2, 'eta': 0.1, 'objective': 'binary:logistic'}
param['eval_metric'] = 'auc'

num_round = 30
evallist = [(dtrain, "train"), (dtest, "test")]
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
prediction_data = pd.read_csv("MLData/LEC_2023_Spring_Test.csv")
# teams = prediction_data[["B_Team", "R_Team"]]
prediction_data = prediction_data.drop(axis = 1, labels = ["B_Team", "R_Team"])

input_vars.remove("WinAsInt")

prediction_data = prediction_data[input_vars]

dpred = xgb.DMatrix(prediction_data)

scores = bst.predict(dpred)
teams["Full Model Scores"] = scores

print(teams)

In [ ]:
teams.to_csv("predictions/shitModel.csv")